<a href="https://colab.research.google.com/github/i-am-akash/Pyspark_Practice/blob/main/sparkdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Spark DataFrame Practice common fns 

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 34.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=12ef31505d658769ff3948bd7a130634069fb76a4d4a83fc504287283c6268bb
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [45]:
from pyspark import SparkContext, SparkConf, sql
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
conf = SparkConf().setAppName("rdd_session")
sc = SparkContext.getOrCreate(conf = conf)
spark = SparkSession.builder.appName("df_session").getOrCreate()

#Reading the file using df

---




In [73]:
df = spark.read.options(inferSchema = "True", header = "True").csv("/content/sample_data/california_housing_test.csv")
df.show()
df.printSchema()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.05|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0|
|   -118.3|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0|
|  -117.81|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|
|  -118.36|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|       6.1359|          330000.0|
|  -119.67|   36.33|              19.0|     1241.0|         244.0|     850.0|     237.0|       2.9375|           81700.0|
|  -119.56|   36.51|    

#providing Schema explicilty

In [41]:
schema = StructType([
  StructField("longitude", DoubleType(), True),
  StructField("latitude", DoubleType(), True),
  StructField("housing_median_age", StringType(), True),
  StructField("total_rooms",StringType(), True),
  StructField("total_bedrooms", StringType(), True),
  StructField("population", StringType(), True),
  StructField("households", StringType(), True),
  StructField("median_income", DoubleType(), True),
  StructField("median_house_value", DoubleType(), True)
])

In [48]:
df_schema = spark.read.options(header = "True").schema(schema).csv("/content/sample_data/california_housing_test.csv")
df_schema = df_schema.withColumn("total_rooms", col("total_rooms").cast("Int"))
df_schema.show()
df_schema.printSchema()

+---------+--------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  -122.05|   37.37|         27.000000|       3885|    661.000000|1537.000000| 606.000000|       6.6085|          344700.0|
|   -118.3|   34.26|         43.000000|       1510|    310.000000| 809.000000| 277.000000|        3.599|          176500.0|
|  -117.81|   33.78|         27.000000|       3589|    507.000000|1484.000000| 495.000000|       5.7934|          270500.0|
|  -118.36|   33.82|         28.000000|         67|     15.000000|  49.000000|  11.000000|       6.1359|          330000.0|
|  -119.67|   36.33|         19.000000|       1241|    244.000000| 850.000000| 237.000000|       2.9375|           81700.0|
|  -119.

#Created DF from RDD 

In [60]:
rdd = sc.textFile("/content/sample_data/california_housing_test.csv")
header = rdd.first()
columns = header.split(",")
rdd = rdd.filter(lambda x : x!=header).map(lambda x : x.split(","))
df_rdd = rdd.toDF(columns)
df_rdd.show()


+-----------+----------+--------------------+-------------+----------------+------------+------------+---------------+--------------------+
|"longitude"|"latitude"|"housing_median_age"|"total_rooms"|"total_bedrooms"|"population"|"households"|"median_income"|"median_house_value"|
+-----------+----------+--------------------+-------------+----------------+------------+------------+---------------+--------------------+
|-122.050000| 37.370000|           27.000000|  3885.000000|      661.000000| 1537.000000|  606.000000|       6.608500|       344700.000000|
|-118.300000| 34.260000|           43.000000|  1510.000000|      310.000000|  809.000000|  277.000000|       3.599000|       176500.000000|
|-117.810000| 33.780000|           27.000000|  3589.000000|      507.000000| 1484.000000|  495.000000|       5.793400|       270500.000000|
|-118.360000| 33.820000|           28.000000|    67.000000|       15.000000|   49.000000|   11.000000|       6.135900|       330000.000000|
|-119.670000| 36.330

Another Way : 

In [64]:
schema = StructType([
  StructField("longitude", StringType(), True),
  StructField("latitude", StringType(), True),
  StructField("housing_median_age", StringType(), True),
  StructField("total_rooms",StringType(), True),
  StructField("total_bedrooms", StringType(), True),
  StructField("population", StringType(), True),
  StructField("households", StringType(), True),
  StructField("median_income", StringType(), True),
  StructField("median_house_value", StringType(), True)
])

In [65]:
df_rdd_1 = spark.createDataFrame(rdd,schema=schema)
df_rdd_1.show()

+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-122.050000|37.370000|         27.000000|3885.000000|    661.000000|1537.000000| 606.000000|     6.608500|     344700.000000|
|-118.300000|34.260000|         43.000000|1510.000000|    310.000000| 809.000000| 277.000000|     3.599000|     176500.000000|
|-117.810000|33.780000|         27.000000|3589.000000|    507.000000|1484.000000| 495.000000|     5.793400|     270500.000000|
|-118.360000|33.820000|         28.000000|  67.000000|     15.000000|  49.000000|  11.000000|     6.135900|     330000.000000|
|-119.670000|36.330000|         19.000000|1241.000000|    244.000000| 850.000000| 237.000000|     2.937500|    

#Selecting Data Frame with Columns :

In [83]:
df = df.withColumn("longitude", col("longitude").cast("String"))
df.printSchema()
df.withColumn("longitude", col("longitude")).show()

root
 |-- longitude: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|             57.95|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0|
|              61.7|   34.26|              43.0|     1510.0|         31

#Create New COlumn 

In [92]:

df_new_col = df.withColumn("personal_room", col("total_rooms")/col("total_bedrooms"))
df_new_col.show()

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|     personal_room|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|27.950000000000003|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0| 5.877458396369137|
|31.700000000000003|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0| 4.870967741935484|
|             32.19|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|7.0788954635108485|
|             31.64|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|   

#Renamed columns with columnRenamed : 

In [93]:
df_new_col = df_new_col.withColumnRenamed("personal_room", "private_room")
df_new_col.show()

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|      private_room|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|27.950000000000003|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0| 5.877458396369137|
|31.700000000000003|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0| 4.870967741935484|
|             32.19|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|7.0788954635108485|
|             31.64|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|   

In [96]:
df_new_col.select(col('private_room').alias("alia")).show()

+------------------+
|              alia|
+------------------+
| 5.877458396369137|
| 4.870967741935484|
|7.0788954635108485|
| 4.466666666666667|
| 5.086065573770492|
| 4.779342723004695|
| 4.484444444444445|
| 4.904458598726115|
| 4.991896272285251|
| 3.800632911392405|
|3.9036144578313254|
| 4.433734939759036|
| 5.983516483516484|
| 5.671469740634006|
| 6.452307692307692|
|             4.025|
| 4.634146341463414|
| 5.069192751235585|
| 8.112244897959183|
| 6.743687834736037|
+------------------+
only showing top 20 rows



In [97]:
df_new_col.filter(col("latitude") == 37.37).show()

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|      private_room|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|27.950000000000003|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0| 5.877458396369137|
|28.120000000000005|   37.37|              14.0|     6016.0|        1404.0|    3258.0|    1316.0|       3.5745|          333700.0| 4.284900284900285|
|27.870000000000005|   37.37|              30.0|     2139.0|         260.0|     742.0|     242.0|       11.806|          500001.0| 8.226923076923077|
|28.189999999999998|   37.37|              26.0|     2987.0|         539.0|    1931.0|     518.0|   

In [98]:
df_new_col.filter((col("latitude") == 37.37)& (col("total_rooms") == 1446.0)).show()

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|     private_room|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------------+
|27.879999999999995|   37.37|              37.0|     1446.0|         181.0|     549.0|     190.0|      10.7355|          500001.0|7.988950276243094|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----------------+



In [102]:
df_new_col.filter(df_new_col.latitude.startswith("3")).show()

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|      private_room|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|27.950000000000003|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0| 5.877458396369137|
|31.700000000000003|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0| 4.870967741935484|
|             32.19|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|7.0788954635108485|
|             31.64|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|   

In [104]:
df_new_col.filter(df_new_col.latitude.endswith("3")).show()

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|      private_room|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|             30.33|   36.33|              19.0|     1241.0|         244.0|     850.0|     237.0|       2.9375|           81700.0| 5.086065573770492|
|28.569999999999993|   38.63|              43.0|     1009.0|         225.0|     604.0|     218.0|       1.6641|           67000.0| 4.484444444444445|
|             32.03|   33.73|              27.0|     2097.0|         325.0|    1217.0|     331.0|       5.7121|          222500.0| 6.452307692307692|
|29.189999999999998|   37.53|              15.0|      570.0|         123.0|     189.0|     107.0|   

In [105]:
df_new_col.filter(df_new_col.latitude.contains("3")).show()

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|      private_room|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|27.950000000000003|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0| 5.877458396369137|
|31.700000000000003|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0| 4.870967741935484|
|             32.19|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|7.0788954635108485|
|             31.64|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|   

In [107]:
df_new_col.filter(df_new_col.latitude.like("%1")).show()

+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|         longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|      private_room|
+------------------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------+
|30.439999999999998|   36.51|              37.0|     1018.0|         213.0|     663.0|     204.0|       1.6635|           67000.0| 4.779342723004695|
|32.010000000000005|   33.81|              42.0|      161.0|          40.0|     157.0|      50.0|          2.2|          153100.0|             4.025|
|31.120000000000005|   34.21|              26.0|     1590.0|         196.0|     654.0|     199.0|       6.5851|          300000.0| 8.112244897959183|
|27.409999999999997|   38.01|              35.0|     8814.0|        1307.0|    3450.0|    1258.0|   